# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [3]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [4]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [6]:
print(os.path.exists('/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/data'))

True


In [7]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 23 images---


  0%|          | 0/19 [00:00<?, ?it/s]

------ Training data created with 80% split 19 images -------


  0%|          | 0/4 [00:00<?, ?it/s]

------ Testing data created with a total of 4 images ----------
------ TASK COMPLETED -------


In [8]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


Training

In [9]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/dataset.yaml epochs=150 imgsz=640 batch=8 project=/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/training_results name=door_and_window

100% 21.5M/21.5M [00:00<00:00, 278MB/s]
Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/dataset.yaml, epochs=150, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/training_results, name=door_and_window, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classe

Inferencing

In [10]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/training_results/door_and_window/weights/best.pt conf=0.55 source=/content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/118_Color.png: 480x640 3 doorss, 133.3ms
image 2/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/29_Color.png: 480x640 4 doorss, 13.1ms
image 3/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/2_Color.png: 480x640 6 doorss, 13.0ms
image 4/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/42_Color.png: 480x640 6 doorss, 13.1ms
image 5/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/46_Color.png: 480x640 (no detections), 13.0ms
image 6/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/51_Color.png: 480x640 3 doorss, 13.1ms
image 7/26 /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/test_images/59_Color.png: 480x640 3 doorss

In [11]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/Door_and_window/yolov8_pointcloud/output